In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.fetch import fetch_sources
from src.preprocess import preprocess
from src.utils import df_na_vals

In [3]:
df = fetch_sources()
df
# preprocess(raw)

,bw_nvlink,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,disk_name,...,pcie_bw,public_ipaddr,reliability,score,static_ip,storage_cost,total_flops,verification,timestamp,country
0,0,890,16,AMD Ryzen 7 5700X 8-Core Processor,63,0,12.2,101,4626,nvme,...,244,96.231.208.215,5727,10,1,150,81,0,1705812798,US
1,0,890,48,AMD EPYC 7402P 24-Core Processor,126,0,12.2,500,7069,,...,239,101.98.36.147,5727,12,1,150,164,0,1705812798,NZ
2,0,610,56,Xeon® E5-2680 v4,47,0,12.2,0,363,P3,...,113,45.61.14.20,5736,1,1,150,15,0,1705812798,CA
3,0,860,16,Xeon® E5-2637 v3,31,0,12.3,0,1941,Seagate,...,118,98.115.241.69,6084,13,1,225,36,0,1705812798,US
4,0,610,56,Xeon® E5-2680 v4,126,0,12.2,0,2186,P3,...,113,45.61.14.20,6325,1,1,150,15,0,1705812798,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,0,890,128,AMD Ryzen Threadripper PRO 3995WX 64-Cores,252,200,12.2,100,3209,GIGABYTE GP-AG42TB,...,128,81.166.173.12,9963,41,1,200,651,1,1705812798,NO
3133,0,860,12,Core™ i7-7800X,125,200,12.2,100,2688,KINGSTON SNV2S4000G,...,117,81.166.162.13,9974,81,1,200,141,1,1705812798,NO
3134,0,610,48,Xeon® E5-2695 v2,126,400,12.0,100,1309,CT500MX500SSD1,...,56,142.112.39.215,9852,56,1,200,114,1,1705812798,CA
3135,0,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1625,Samsung SSD 970 EVO 1TB,...,58,194.44.114.10,9965,61,1,150,49,2,1705812798,UA


In [4]:
cpu_name = df.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = df.cpu_name.str.replace('Core™', '')
df.cpu_name = cpu_name

df['cores'] = (df.cpu_cores / df.num_gpus).fillna(0).astype(int)
df['ram'] = (df.cpu_ram / df.num_gpus).round().astype(int)
df['dph'] = (df.dph_base / df.num_gpus).round().astype(int)
df['disk'] = (df.disk_space / df.num_gpus).astype(int)
df['rented_fraq'] = '[' + df.num_gpus_rented.astype(str) + "/" + df.num_gpus.astype(str) + ']'
# df['rented'] = df.num_gpus_rented > 0
df['rented'] = df.num_gpus_rented == df.num_gpus
# df['min_fraq'] = df.min_chunk/df.num_gpus

# df.dlperf = df.dlperf.round(1)
# df.dlperf_per_dphtotal = df.dlperf_per_dphtotal.round(1)
# df.disk_bw = df.disk_bw.round()
# df.duration = pd.to_timedelta(df.duration * 1e9).dt.days
# df.inet_down = df.inet_down.fillna(0).astype(int)
# df.inet_up = df.inet_up.fillna(0).astype(int)
# df.inet_up_cost = (df.inet_up_cost * 1000).fillna(0).astype(int)
# df.inet_down_cost = (df.inet_down_cost * 1000).fillna(0).astype(int)

# df['isp'] = df.location.apply(lambda x: x.get('isp', None) if not pd.isna(x) else None)

# df.end_date = (pd.to_datetime(df.end_date * 1e9) - pd.Timestamp.now()).dt.days
df.end_date = pd.to_datetime(df.end_date * 1e9)
# df.start_date = pd.to_datetime(df.start_date * 1e9)
# df.set_index('machine_id', inplace=True)

In [5]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'country']

df.query("dph >= 0.5 and gpu_name == 'RTX 4090'")[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
926,RTX 4090,[0/1],2000,400,128,126,1560,2714,1,HR,...,0,12.2,535.146.02,4,16,0,39,AMD EPYC 7742 64C Processor,H12SSL,HR
1472,RTX 4090,[0/2],1000,0,24,126,1670,5727,1,US,...,2,12.2,535.146.02,4,16,0,241,AMD EPYC 7402P 24C Processor,H12SSL,US
1158,RTX 4090,[0/12],1000,600,10,42,555,11264,12,US,...,2,12.2,535.129.03,4,8,0,128,AMD EPYC 7B12 64C Processor,ROMED16QM3,US
307,RTX 4090,[0/1],1000,0,64,126,1500,697,1,US,...,4,12.2,535.129.03,3,16,0,69,AMD EPYC 7551P 32C Processor,H11SSL,US
692,RTX 4090,[2/5],850,50,25,50,1326,6844,1,US,...,10,12.2,535.129.03,4,16,0,198,AMD EPYC 7532 32C Processor,ROME2D32GM,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,RTX 4090,[1/1],293,0,32,78,1530,3199,1,US,...,0,12.2,535.146.02,4,16,0,254,AMD Ryzen 9 5950X 16C Processor,B550 Phantom Gaming 4,US
2039,RTX 4090,[1/1],290,0,12,16,550,838,1,US,...,0,12.0,525.147.05,3,16,0,124,AMD Ryzen 9 3900X 12C Processor,,US
1984,RTX 4090,[1/1],290,400,12,16,650,805,1,US,...,0,12.0,525.147.05,3,16,0,124,AMD Ryzen 9 3900X 12C Processor,,US
1626,RTX 4090,[1/1],290,400,12,31,740,830,1,US,...,0,12.0,525.147.05,3,16,0,124,AMD Ryzen 9 3900X 12C Processor,,US


In [12]:
df.query("num_gpus == 4 and gpu_name == 'RTX 4090'")[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
13,RTX 4090,[0/4],400,0,12,16,367,1964,2,CN,...,2,12.2,535.129.03,3,16,0,117,Xeon® E5-2673 v3,X10DRG,CN
14,RTX 4090,[0/4],400,0,20,78,367,569,1,CN,...,2,12.2,535.129.03,3,16,0,117,Xeon® Gold 6133,X11DPG,CN
15,RTX 4090,[4/4],687,0,64,126,827,6072,4,US,...,31,12.2,535.146.02,4,8,0,123,AMD EPYC 7B12 64C Processor,ROME2D32GM,US
62,RTX 4090,[0/4],360,0,7,62,45,642,1,US,...,2,12.0,525.147.05,3,16,0,116,Xeon® E5-2660 v3,,US
214,RTX 4090,[0/4],400,0,8,63,212,4812,1,IS,...,0,12.2,535.113.01,4,16,0,239,AMD EPYC 7282 16C Processor,H12SSL,IS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,RTX 4090,[4/4],500,200,12,63,270,4060,4,TW,...,2,12.2,535.113.01,4,16,0,245,AMD EPYC 7402 24C Processor,ROMED8,TW
3014,RTX 4090,[4/4],450,0,12,63,395,3342,4,TW,...,2,12.2,535.113.01,4,16,0,244,AMD EPYC 7402 24C Processor,ROMED8,TW
3060,RTX 4090,[2/4],500,200,16,32,317,3435,1,NO,...,20,12.2,535.129.03,4,16,0,244,AMD Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO
3062,RTX 4090,[1/4],500,200,16,32,310,3374,1,NO,...,20,12.2,535.129.03,4,16,0,240,AMD Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO


In [7]:
df[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
364,RTX 4090,[0/2],395,50,16,63,1665,6010,1,RU,...,1,12.2,535.146.02,4,8,0,128,AMD Ryzen 9 5950X 16C Processor,PRIME X570,RU
444,RTX 4090,[2/2],319,0,6,32,320,784,1,RU,...,2,12.2,535.146.02,4,4,0,61,AMD Ryzen 5 5600 6C Processor,X570 AORUS MASTER,RU
452,RTX 4090,[0/2],390,0,32,63,2680,6206,1,RU,...,0,12.2,535.146.02,4,16,0,239,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
455,RTX 3090,[2/2],162,0,6,32,300,766,1,RU,...,2,12.2,535.146.02,3,2,0,14,AMD Ryzen 5 3600 6C Processor,B550 AORUS ELITE V2,RU
488,RTX 4090,[0/2],410,0,32,63,835,1981,1,RU,...,0,12.2,535.146.02,4,16,0,242,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
530,RTX 4090,[0/2],410,0,32,47,5360,8301,1,RU,...,0,12.2,535.146.02,4,16,0,245,AMD Ryzen TR 3970X 32C Processor,TRX40 AORUS PRO WIFI,RU
533,RTX 3090,[0/1],169,0,12,31,620,1466,1,RU,...,2,12.2,535.146.02,4,16,0,250,AMD Ryzen 5 5600 6C Processor,B550 AORUS ELITE V2,RU
544,RTX 4090,[0/2],395,50,16,63,1670,5932,1,RU,...,1,12.2,535.129.03,4,8,0,126,AMD Ryzen 9 5950X 16C Processor,PRIME X570,RU
545,RTX 4090,[0/2],410,100,32,63,2680,5617,1,RU,...,0,12.2,535.146.02,4,8,0,125,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
611,RTX 3090,[2/2],169,0,8,32,315,1485,1,RU,...,0,12.2,535.113.01,3,16,0,115,Xeon® E5-2667 v4,E5,RU
